# LLM Applications
### 학습 목표
1. Prompt를 활용하여 LLM Application을 제작해 보자.
2. OpenAI API의 Function Calling 기능을 활용해 보자.

***context***
1. LLM Application  
2. Function Calling

## 0. 준비 과정
본 실습에선 정확한 동작 결과를 확인하기 위하여 OpenAI의 GPT-4 (`gpt-4-1106-preview`) 모델을 활용할 것이다.
* `gpt-3.5-turbo-0125`로도 가능하지만, 정확도가 낮으므로 정확한 결과를 기대하기 힘들다.
* OpenAI API 유료 사용자만 GPT-4를 사용할 수 있으므로, 무료 사용자는 `gpt-3.5-turbo-0125`로 동작 원리만을 파악해 보고, 동작 결과는 실습 영상을 참고하거나 추후 GPT-4 동일한 코드를 실행해 보도록 하자.

> Response format

[Response format](https://platform.openai.com/docs/api-reference/chat/create#chat-create-response_format)을 통해 언어 모델의 답변을 구조화하여 저장할 수 있다.

이러한 기능은 ChatGPT의 응답을 통해 데이터를 생성하고 저장하는 상황에서 활용될 수 있다. Response format을 JSON으로 설정하면, ChatGPT의 출력물을 JSON 형태로 확인하고 저장할 수 있다.

In [1]:
!pip install "openai" "tenacity" "duckduckgo_search==5.3.1b1" "html2text"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.7/336.7 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 36.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.5 MB/s eta 0:00:00
  Created wheel for html2text: filename=html2text-2024.2.26-py3-none-any.whl size=33111 sha256=c8587bb6b1408e17832ded1f5023d7246c56d3c3d5ac31697c43baa60b4b486b
  Stored in directory: /root/.cache/pip/wheels/f3/96/6d/a7eba8f80d31cbd188a2787b81514d82fc5ae6943c44777659
Successfully built html2text


In [ ]:
from openai import OpenAI
from tenacity import retry, stop_after_attempt, wait_random_exponential

client = OpenAI(
    api_key="여기에 OPENAI_API_KEY를 입력하세요."
)

class MySession:
    def __init__(
        self,
        system_prompt: str="", # 모델의 응답 방식을 정해주거나, 페르소나를 입힐 수 있다.
        model="gpt-4-1106-preview", # 무료 사용자는 "gpt-3.5-turbo-0125" 모델을 사용한다.
        is_json_output=True
    ) -> None:
        self.model = model
        self.is_json_output = is_json_output
        self.messages = [{"role": "system", "content": system_prompt}] if system_prompt else []

    @retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
    def chat(self, query: str) -> str:
        self.messages.append({"role": "user", "content": query}) # 입력 query를 messages에 추가한다.
        response = client.chat.completions.create( # response를 생성한다.
            model=self.model,
            messages=self.messages,
            response_format={"type": "json_object"} if self.is_json_output else None # json_format으로 모델의 응답을 반환할 수 있다.
        ).choices[0].message
        self.messages.append(response) # 모델의 response를 messages에 추가
        return response.content # 모델의 response를 반환한다.

    def revert_last_chat(self) -> None: # API 호출에 오류가 생겼을 경우를 대비하기 위한 메소드이다.
        if len(self.messages) > 2:
            self.messages = self.messages[:-2]

    def get_last_chat(self) -> None: # 마지막 유저의 입력과 모델의 응답을 반환한다.
        return self.messages[-2:]

In [ ]:
session = MySession(is_json_output=False)
print(session.chat("What is Deep Learning? Please summarize in one sentence."))

Deep learning is a subset of machine learning that employs complex neural networks to model and find patterns in large datasets, enabling systems to make decisions with minimal human intervention.


In [ ]:
session = MySession(is_json_output=True) # JSON 형태로 결과물을 확인해 보자.
output = session.chat("What is Deep Learning? Please summarize in one sentence in a json string!.") # json모드로 출력을 생성하려면 입력 문자열에 "json"이 포함되어야 한다.
print("Type:", type(output))
print("output:", output)

Type: <class 'str'>
output: {
  "summary": "Deep Learning is a subset of machine learning involving neural networks with multiple layers to model complex patterns in data."
}


In [ ]:
session.messages

[{'role': 'user',
  'content': 'What is Deep Learning? Please summarize in one sentence in a json string!.'},
 ChatCompletionMessage(content='{\n  "summary": "Deep Learning is a subset of machine learning involving neural networks with multiple layers to model complex patterns in data."\n}', role='assistant', function_call=None, tool_calls=None)]

## 1. ChatGPT를 활용한 LLM Application
먼저, OpenAI API를 통해 활용할 수 있는 3가지 기능들을 정의해보자.
- web_search_engine
- question_answering
- math_calculator

위 기능들뿐만 아니라, 본인이 개발하는 Application에 사용하고자 하는 기능들을 정의하는 것 역시 가능하다.

### web_search_engine
인터넷에서 사용자가 입력한 쿼리를 검색하는 기능이다. 본 실습에선 검색 API를 활용하여 검색 결과를 가져온다.
* [DuckDuckGo](https://duckduckgo.com/)에서 제공하는 검색 API를 활용하여 쿼리에 따른 응답을 검색해 본다.
* 검색된 상위 5개의 결과를 JSON 텍스트 형태로 출력한다.

In [ ]:
from duckduckgo_search import DDGS
import json

def web_search_engine(query: str):
    with DDGS() as ddgs:
        return json.dumps([r for r in ddgs.text(query, max_results=5)])

In [ ]:
result = web_search_engine("What are the recent events in South Korea?")
print(result)

[{"title": "South Korea - BBC News", "href": "https://www.bbc.com/news/topics/cnx753jej1xt", "body": "South Korea sack Klinsmann after a year in charge. Jurgen Klinsmann is sacked as head coach of South Korea after just 12 months in the role following their Asian Cup exit. Football. 15 Feb 2024."}, {"title": "South Korea | AP News", "href": "https://apnews.com/hub/south-korea", "body": "South Korea's presidential office claims that the country's move to establish diplomatic relations with Cuba would deal a \"political and psychological blow\" to its war-divided rival North Korea, whose diplomatic footing is largely dependent on a small number of Cold War allies. Updated 2:29 AM PST, February 15, 2024."}, {"title": "South Korea | Today's latest from Al Jazeera", "href": "https://www.aljazeera.com/where/south-korea/", "body": "North Ko\u00adrea ends all eco\u00adnom\u00adic co\u00adop\u00ader\u00ada\u00adtion with South as ties hit new low. Re\u00adla\u00adtions have de\u00adte\u00adri\u

### question_answering
모델이 주어진 URL에 포함된 텍스트를 읽어 질문에 답변하는 기능을 정의해보자.
* 해당 함수를 사용하지 않는다면, 주어진 URL에서 텍스트를 출력하여 별도의 변수에 저장하고, 해당 변수를 GPT-4에 직접 입력하여 답변을 생성해야 할 것이다. 이는 번거로움을 야기한다.
* 따라서 해당 동작을 동일하게 수행하는 별도의 함수를 만듦으로써 GPT-4에 직접 URL 크롤링 결과를 입력하지 않고 자동화할 수 있다.
* 해당 함수에선 기계 독해를 진행하기 위하여 더 긴 Text를 다룰 수 있고 GPT-4에 비해 가벼운 `gpt-3.5-turbo-16k`를 활용한다.

In [ ]:
from urllib.request import Request, urlopen
import html2text

def question_answering(url: str, question: str, model="gpt-3.5-turbo-16k"):
    html_converter = html2text.HTML2Text() # HTML을 text로 변환하는 html_converter 생성
    html_converter.ignore_links = True # link를 무시하여 clean한 텍스트를 생성할 수 있다.

    request = Request(url, headers={'User-Agent': 'Mozilla/5.0'}) # url을 입력하여 request를 생성한다.
    with urlopen(request) as response:
        webpage_text = response.read().decode() # webpage content를 string형태로 decode한다.
        webpage_text = html_converter.handle(webpage_text) # HTML content를 text로 변환한다.

    context = "Read the following context and answer the question\n\n# Context\n {webpage_text}"
    session = MySession(
        context.format(webpage_text=webpage_text),
        model=model,
        is_json_output=False
    )
    answer = session.chat(question)
    return session.chat(question)

In [ ]:
args = {
    "url": "https://www.bbc.com/news/topics/cnx753jej1xt/",
    "question": "What are the recent events in South Korea?"
}
print(question_answering(**args))

1. A K-pop star publicly apologizes after confirming her relationship with actor Lee Jae-wook.
2. Junior doctors in South Korea have gone on strike, leading to disruptions and delays in surgeries.
3. South Korea is facing a low birth rate, despite spending billions to reverse the trend. Young women's needs are said to be overlooked in the government's efforts.
4. An ongoing doctor strike in South Korea resulted in a woman in her 80s being turned away from multiple hospitals and subsequently passing away.
5. Son Heung-min, captain of the South Korea national football team, appeals to fans to forgive his teammate Lee Kang-in following an altercation between them at the Asian Cup.
6. Surgeries in South Korea have been delayed as medical interns and residents resign or go on strike in response to a recruitment proposal.
7. Jurgen Klinsmann, head coach of the South Korea national football team, gets sacked just one year into the role after their exit from the Asian Cup.
8. A rescuer known a

### math_calculator
주어진 수식을 Python으로 직접 계산한다. 물론 외부 계산기 API 등도 충분히 사용 가능하다.

In [ ]:
def math_calculator(equation: str):
    return str(eval(equation)) # eval 함수는 문자열로 들어온 수식을 연산하는 파이썬 내장 함수이다.

In [ ]:
print(math_calculator("15 / 2"))

7.5


### Main Prompt
web_search_engine, question_answering, math_calculator 함수들을 정의하였다.
이 함수들을 ChatGPT가 활용할 수 있게끔 아래와 같이 Prompt를 구성한다.
* JSON 생성 과정 중에 `thought`를 생성하여 모델의 사고 과정을 들여다보고, 이를 토대로 Chain-of-Thought 및 Planning을 진행해 보자.
* JSON 형태로 생성하게 하여 출력물을 더욱 쉽게 저장하고 활용할 수 있다.
* `terminate` 명령어를 정의하여 ChatGPT가 최종 답변을 출력할 수 있게 한다.
* 아래의 Prompt는 [AutoGPT](https://github.com/Significant-Gravitas/AutoGPT)를 참고하였다.

In [ ]:
SYSTEM_PROMPT = """\
You are searchGPT, a professional tool that fetches the latest information from the internet.
Your decisions must always be made independently without seeking user assistance.
Play to your strengths as an LLM and pursue simple strategies with no legal complications.

## Commands
These are the ONLY commands you can use.
Any action you perform must be possible through one of these commands:
1. web_search_engine: Searches the web with the given query. Params: (query: string)
2. question_answering: Refer to the given URL, and answer a simple question. Params: (url: string, question: string)
3. math_calculator: Calculate the given equation. The equation must be evaluable in Python. Params: (equation: string)
4. terminate: Finally, response to the the user's question. If the user asked in Korean, then answer in Korean. Params: (answer: string)

## Your Task
The user will specify a task for you to execute, in triple quotes, in the next message.
Your job is to complete the task while following your directives as given above, and terminate when your task is done.

Respond strictly with a JSON object.
The JSON object should be compatible with the TypeScript type `Response` from the following:
interface Response {
    thought: {
        // Relevant observations from your last action (if any)
        observations?: string;
        // Thoughts
        text: string;
        reasoning: string;
        // Short markdown-style bullet list that conveys the long-term plan
        plan: string;
    }
    command: {
        name: string;
        args: {
            [param: string]: any
        };
    }
}

Determine exactly one command to use next based on the given goals and the progress you have made so far, and respond using the JSON schema specified previously.\
"""


In [ ]:
# 위 시스템 프롬프트를 기반으로 Main Prompt를 구현한다.

MAX_TURN = 10

def main_prompt(query: str):
    main_session = MySession(SYSTEM_PROMPT) # 세션 생성
    for turn in range(1, MAX_TURN + 1): # 사전에 입력한 최대 TURN만 반복한다.
        print(f"\n### Turn {turn}")
        for attempt in range(3): # 최대 3번만 시도한다.
            try:
                print("query:", query)
                model_output = main_session.chat(query) # query에 따른 모델의 응답
                response = json.loads(model_output) # 모델의 output을 load한다.

                if "observations" in response["thought"]:
                    print("ChatGPT's' observation:", response["thought"]["observations"]) # 모델의 관찰
                print("ChatGPT's thought:", response["thought"]["text"]) # 모델의 생각
                print("ChatGPT's reasoning:", response["thought"]["reasoning"]) # 모델의 사고
                print("ChatGPT's plan:\n", response["thought"]["plan"], sep='') # 모델의 계획
                print("ChatGPT's command:", json.dumps(response["command"])) # 계획을 실행하기 위해 필요한 command. 특정 command를 실행하기 위한 arg도 함께 생성된다.

                match response["command"]["name"]:
                    case "web_search_engine":
                        output = web_search_engine(**response["command"]["args"]) # web_search_engine을 통해 결과를 가져온다.
                    case "question_answering":
                        output = question_answering(**response["command"]["args"]) # question_answering을 통해 결과를 가져온다.
                    case "math_calculator":
                        output = math_calculator(**response["command"]["args"]) # math_calculator를 통해 결과를 가져온다.
                    case "terminate": # 함수를 종료한다.
                        return response["command"]["args"]["answer"]
                    case _:
                        raise NotImplementedError()
                print("ChatGPT's output after running the command", output) # command를 실행한 이후 output을 출력한다.
                query = output # command 실행 결과를 query에 할당하여 line12의 main_session.chat에 입력한다.

            except:
                print(f"에러가 발생했습니다. 재생성 기회가 {2 - attempt}회 남았습니다.")
                main_session.revert_last_chat() # main_session.messages에 누적되어 있는 마지막 기록을 제거한 이후 다시 try한다.

            else:
                break
        else:
            print("답변 생성에 실패하여, 생성을 중단합니다.")
            raise ValueError()

In [ ]:
response = main_prompt("2023년 한국에서 발생한 사건들을 요약해줘.")
print("\nChatGPT의 최종 답안:", response)


### Turn 1
query: 2023년 한국에서 발생한 사건들을 요약해줘.
ChatGPT's thought: I need to search the web for a summary of events that occurred in South Korea in 2023.
ChatGPT's reasoning: The user has asked for a summary of events in South Korea for the current year, 2023; to provide an accurate and up-to-date response, I need to perform a web search.
ChatGPT's plan:
- Perform a web search for events in South Korea in 2023
- Review the search results
- Provide a summary of the findings
ChatGPT's command: {"name": "web_search_engine", "args": {"query": "2023 South Korea events summary"}}
ChatGPT's output after running the command [{"title": "2023 in South Korea - Wikipedia", "href": "https://en.wikipedia.org/wiki/2023_in_South_Korea", "body": "The following lists events in the year 2023 in South Korea. Incumbents. Office Image Name Tenure / Current length President of the Republic of Korea: Yoon Suk Yeol: 10 May 2022 (21 months ago) () Speaker of the National Assembly: Kim Jin-pyo: 4 July 2022 (19 mont

## Function Calling

OpenAI API에선 도구 사용을 지원하기 위해, [Function calling](https://platform.openai.com/docs/guides/function-calling/function-calling) 기능을 가지고 있다. Function calling을 활용하여 앞서 만든 기능을 구현해 보자.

In [ ]:
# tools 내에 dictionary 형태로 함수들을 정의한다.

tools = [
    {
        "type": "function",
        "function": {
            "name": "web_search_engine",
            "description": "Perform a web search using the given query and output the content",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "A search query",
                    }
                },
                "required": ["query"],
            },
        },
    },
     {
        "type": "function",
        "function": {
            "name": "question_answering",
            "description": "Generate an answer to the question based on the given URL",
            "parameters": {
                "type": "object",
                "properties": {
                    "url": {
                        "type": "string",
                        "description": "Target URL"
                    },
                    "question": {
                        "type": "string",
                        "description": "A question to ask based on URL content",
                    }
                },
                "required": ["url", "question"],
            },
        },
    },
     {
        "type": "function",
        "function": {
            "name": "math_calculator",
            "description": "Computes the given equation and returns the result",
                "type": "object",
                "properties": {
                    "equation": {
                        "type": "string",
                        "description": "A equation evaluable in Python"
                    }
                },
                "required": ["equation"],
            },
      },
]

In [ ]:
available_functions = {
    "web_search_engine": web_search_engine,
    "question_answering": question_answering,
    "math_calculator": math_calculator
}
MAX_TURN = 10

def main_prompt_function_calling(query: str):
    print("질문:", query, end="") # 사용자의 입력
    messages = [
        {"role": "system", "content": "If the user asked in Korean, then answer in Korean."},
        {"role": "user", "content": query}
    ]

    for turn in range(1, MAX_TURN + 1): # 사전에 입력한 최대 TURN만 반복한다.
        print(f"\n### Turn {turn}")
        for attempt in range(3): # 최대 3번만 시도한다.
            try:
                response = client.chat.completions.create(
                    model="gpt-4-1106-preview", # 또는 gpt-3.5-turbo-0125를 사용한다.
                    messages=messages,
                    tools=tools, # 앞서 정의한 tools를 입력한다.
                )
                response_message = response.choices[0].message # 모델의 결과
                print("모델의 응답:", response_message)
                if response_message.tool_calls: # 함수를 사용해야 할 상황이라면 reponse_message의 tool_calls에 사용할 함수와 argument가 함께 반환된다.
                                                # 즉, tools를 입력하더라도 tool이 필요 없다면 바로 답변을 생성하는 경우도 있다.
                    new_messages = [response_message]
                    for tool_call in response_message.tool_calls:
                        print("ChatGPT's function:", tool_call.function.name, tool_call.function.arguments) # ChatGPT가 사용할 function과 argument
                        print("Output after running the function", content := available_functions[tool_call.function.name](**json.loads(tool_call.function.arguments)))
                        new_messages.append({ # 사용한 tool과 그에 따른 결과(content)를 dictionary 형태로 append한다.
                            "tool_call_id": tool_call.id,
                            "role": "tool",
                            "name": tool_call.function.name, # 사용한 function을 입력한다.
                            "content": content # function을 실행할 결과를 입력한다.
                        })
                    messages.extend(new_messages) # message에 추가한다.
                else:
                    return response_message.content

            except Exception as e:
                print(e)
                print(f"에러가 발생했습니다. 재생성 기회가 {2 - attempt}회 남았습니다.")
            else:
                break
        else:
            print("답변 생성에 실패하여, 생성을 중단합니다.")
            raise ValueError()

In [ ]:
response = main_prompt_function_calling("2023년 한국에서 발생한 사건들을 요약해줘.")
print("ChatGPT의 최종 답안:", response)

질문: 2023년 한국에서 발생한 사건들을 요약해줘.
### Turn 1
모델의 응답: ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_LDm8HQZeJqFLYIyheBCWPJaF', function=Function(arguments='{"query":"2023년 한국 사건 요약"}', name='web_search_engine'), type='function')])
ChatGPT's function: web_search_engine {"query":"2023년 한국 사건 요약"}
Output after running the function [{"title": "10\ub300 \uad6d\ub0b4\ub274\uc2a4 | 2023 10\ub300\ub274\uc2a4 | \uc5f0\ud569\ub274\uc2a4", "href": "https://www.yna.co.kr/2023-site/year-end/national", "body": "\uc5f0\ud569\ub274\uc2a4 \uc120\uc815. 10\ub300 \uad6d\ub0b4\ub274\uc2a4. 2023\ub144 \uc815\uce58\uad8c\uc5d0\uc11c\ub294 \uc9c4\uc601 \uc815\uce58\uac00 \uc2ec\ud654\ud558\uba74\uc11c \uc5ec\uc57c \uac04 \uadf9\ud55c \ub300\uce58\uac00 \uacc4\uc18d\ub410\ub2e4. \uace0\ubb3c\uac00 \uc18d \ud2b9\ud788 \uc2dd\ud488 \ubb3c\uac00\uac00 \ud06c\uac8c \uc62c\ub77c \uc11c\ubbfc\ub4e4\uc758 \uc0b6\uc774 \ud30d\ud30d\ud574\uc84c\